<div style="float:left">
    <h1 style="width:450px">Practical 10: </h1>
    <h2 style="width:450px">Linking &amp; Visualising Data</h2>
</div>
<div style="float:right"><img width="100" src="https://github.com/jreades/i2p/raw/master/img/casa_logo.jpg" /></div>

| Complete | Part 1: Foundations | Part 2: Data | Part 3: Analysis |     |
| :------- | :------------------ | :----------- | :--------------- | --: |
| 90% | &#9619;&#9619;&#9619;&#9619;&#9619;&#9619;&#9619;&#9619; | &#9619;&#9619;&#9619;&#9619;&#9619;&#9619; | &#9619;&#9619;&#9619;&#9619;&#9619;&#9617; | 10/10

<div class="alert alert-block alert-warning">
    <b>&#9888; Important</b>: This practical focusses on two key bits of <em>implementation</em>: visualisation and data linkage! You will have seen quite a bit of each of these across the preceding three to four weeks, but they were picked up in an <em>ad-hoc</em> way, here we try to systematise things a bit.</div>

<div class="alert alert-block alert-info">
    <b>&#128279; Connections</b>: Here we're trying to tidy up the loose ends. You've already worked with basic data visualisations in Seaborn and Matplotlib (including (geo)panda's <tt>plot</tt> function), but we want you to have a better sense of how that <em>works</em> as part of a coherent -- if altogether rather complex and overwhelming -- approach to managing a data visualisation. You've also already seen examples of joins and spatial joins before but, again, we just want to review them more formally now.
</div>

### Required Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns

import matplotlib.cm as cm
import matplotlib.pyplot as plt

In [ ]:
import os
from requests import get
from urllib.parse import urlparse

def cache_data(src:str, dest:str) -> str:
    """Downloads and caches a remote file locally.
    
    The function sits between the 'read' step of a pandas or geopandas
    data frame and downloading the file from a remote location. The idea
    is that it will save it locally so that you don't need to remember to
    do so yourself. Subsequent re-reads of the file will return instantly
    rather than downloading the entire file for a second or n-th itme.
    
    Parameters
    ----------
    src : str
        The remote *source* for the file, any valid URL should work.
    dest : str
        The *destination* location to save the downloaded file.
        
    Returns
    -------
    str
        A string representing the local location of the file.
    """
    
    url = urlparse(src) # We assume that this is some kind of valid URL 
    fn  = os.path.split(url.path)[-1] # Extract the filename
    dfn = os.path.join(dest,fn) # Destination filename
    
    # Check if dest+filename does *not* exist -- 
    # that would mean we have to download it!
    if not os.path.isfile(dfn) or os.path.getsize(dfn) < 1:
        
        print(f"{dfn} not found, downloading!")

        # Convert the path back into a list (without)
        # the filename -- we need to check that directories
        # exist first.
        path = os.path.split(dest)
        
        # Create any missing directories in dest(ination) path
        # -- os.path.join is the reverse of split (as you saw above)
        # but it doesn't work with lists... so I had to google how
        # to use the 'splat' operator! os.makedirs creates missing
        # directories in a path automatically.
        if len(path) >= 1 and path[0] != '':
            os.makedirs(os.path.join(*path), exist_ok=True)
            
        # Download and write the file
        with open(dfn, "wb") as file:
            response = get(src)
            file.write(response.content)
            
        print('Done downloading...')

    else:
        print(f"Found {dfn} locally!")

    return dfn

## Task 1. Spatial Joins (Recap)

### Task 1.1: Load Geodata

A lot of useful geo-data can be accessed from the [GeoPortal](https://geoportal.statistics.gov.uk/). And see also [my discussion](https://jreades.github.io/fsds/lectures/9.2-Linking_Spatial_Data.html#/think-it-through) on [lookup tables](https://geoportal.statistics.gov.uk/datasets/postcode-to-output-area-to-lower-layer-super-output-area-to-middle-layer-super-output-area-to-local-authority-district-november-2018-lookup-in-the-uk-2/about).

In [ ]:
spath = 'https://github.com/jreades/fsds/blob/master/data/src/' # source path
ddir  = os.path.join('data','geo') # destination directory
water = gpd.read_file( cache_data(spath+'Water.gpkg?raw=true', ddir) )
boros = gpd.read_file( cache_data(spath+'Boroughs.gpkg?raw=true', ddir) )
green = gpd.read_file( cache_data(spath+'Greenspace.gpkg?raw=true', ddir) )

In [ ]:
msoas = gpd.read_file( cache_data(spath+'Middle_Layer_Super_Output_Areas__December_2011__EW_BGC_V2-shp.zip?raw=true', ddir) )
msoas.plot();

In [ ]:
msoas.sample(3)

### Task 1.2: Select London MSOAs

<div class="alert alert-block alert-info">
    <b>&#128279; Connections</b>: One thing to remember here is that computers are <em>exact</em>. So if you say that the selection should only be of MSOAs <em>within</em> London then you actually need to think about whether a shared border qualifies as 'within'. Watch <a href="https://jreades.github.io/fsds/sessions/week10.html#lectures">the lectures</a> again if you're unsure, but that's why here we take this slightly clunk approach of buffering the London boundary <em>before</em> doing the selection.
</div>

#### 1.2.1. Union

As we don't have a boundary file for London, we can *generate* use using the `unary_union` operator (as we do here) or using the [dissolve()](https://geopandas.org/en/stable/docs/user_guide/aggregation_with_dissolve.html) approach. Consider the pros and cons of each approach in terms of performance, output format, and leigibility. 

So here's approach 1, which is a function call (on which we call `plot`):

In [ ]:
boros.dissolve().plot();

And here's approach 2, which is an *attribute* and returns a polygon (so no reason to call `plot`, but it's come back without the rest of the data frame!):

In [ ]:
boros.unary_union

<div class="alert alert-block alert-info">
    <b>&#128279; Connections</b>: Notice how we're also demonstrating some additional ways of plotting 'on the fly' (without generating a data frame) as well as reminding you how to zoom in/out.
</div>

In [ ]:
ldn = gpd.GeoDataFrame(gpd.GeoSeries(data=boros.unary_union)).rename(columns={0:'geometry'}).set_geometry("geometry")
ldn = ldn.set_crs(epsg=27700)
ax  = ldn.plot(facecolor=(.5, .5, .9, .5));
msoas.plot(ax=ax, facecolor='none', edgecolor=(.6, .6, .6, .6))
ax.set_xlim(500000, 515000)
ax.set_ylim(180000, 195000);

#### 1.2.2. Buffer

In order to ensure that we get all MSOAs _within_ London we will buffer the boundary by 250m. If _cover_ were easier to use then that option might be preferable.

In [ ]:
ldn['buffered'] = ldn.geometry.???(???)
ldn = ldn.set_geometry('buffered').set_crs(epsg=27700)
ax  = ldn.plot(facecolor=(.5, .5, .9, .5))
msoas.plot(ax=ax, facecolor='none', edgecolor=(.6, .6, .6, .6))
ax.set_xlim(500000, 515000)
ax.set_ylim(180000, 195000);

#### 1.2.3. Spatial Join

Here's our first spatial join. By default it will be an _inner_ join because we want to drop everything that doesn't line up between the two data sets (i.e. don't keep the thousands of *other* MSOAs).

In [ ]:
ldn_msoas = gpd.sjoin(msoas, ldn, predicate='???', how='inner')
ldn_msoas.head(2)

#### 1.2.4. Plot

In [ ]:
ldn_msoas.plot()

Hmmmm, not quite what you were expecting? See if you can figure out from the list of columns and the documentation for `set_geometry` what is going wrong?

In [ ]:
ax = ldn_msoas.set_geometry('geometry_left').plot(linewidth=2, facecolor='none', edgecolor=(.6, .6, .6, .6))
ax.set_xlim(500000, 515000)
ax.set_ylim(180000, 195000);

In [ ]:
ldn_msoas = ldn_msoas.rename(columns={'geometry_left':'geometry'}).set_geometry('geometry')

In [ ]:
ldn_msoas.drop(columns='geometry_right', inplace=True)

We no longer really need to keep the full MSOA data set hanging about.

In [ ]:
try:
    del(msoas)
except NameError:
    print("msoas already deleted.")

#### 1.2.5. Questions

- Can you explain *why* the outputs of the `dissolve` and `unary_union` *look* differnet? And use that as the basis for explaining why they *are* different?

> 

- How do you know that the units for the buffering operation are metres? 250 could be *anything* right?

> 

- Why do we need to buffer the London geometry *before* performing the *within* spatial join?

> 

### Task 1.3: Append Names

We don't actually make use of these in this session, but *both* operations could be relevant to your final reports:

1. The Borough > Subregion mapping could help you to group your data into larger sets so that your resulst become more reobust. it also connects us to long-run patterns of socio-economic development in London.
2. The MSOA Names data set gives you something that you could use to label one or more 'neighbourhoods' on a map with names that are *relevant*. So rather than talking about "As you can see, Sutton 003, is...", you can write "The Wrythe neighbourhood [or area] of Sutton is significantly different from the surrounding areas..."

They also usefully test your understanding of regular expressions and a few other aspects covered in previous weeks.

#### 1.3.1. Replace

You've done this before: notice that the MSOA Name _contains_ the Borough name **with a space and some digits at the end**. Use a regex (in `str.replace()`) to extract the LA name from the MSOA name. See if you do this *without* having to find your previous answer!

In [ ]:
ldn_msoas['Borough'] = ldn_msoas.MSOA11NM.str.replace(r'???','',regex=True)

# Just check results look plausible; you should have:
# - 33 boroughs
# - A df shape of 983 x 13
print(ldn_msoas.Borough.unique())
print(f"There are {len(ldn_msoas.Borough.unique())} boroughs.")
print(f"Overall shape of data frame is {' x '.join([str(x) for x in ldn_msoas.shape])}")

#### 1.3.2. Merge

The House of Commons Library provides a [MSOA Names](https://houseofcommonslibrary.github.io/msoanames/) data set that contains locally-relevant names applied to MSOAs. These seek to connect the Census geography (OA > LSOA > MSOA > LA) to a loosely-defined 'neighbourhood'.

In [ ]:
msoa_nms = pd.read_csv( cache_data('https://houseofcommonslibrary.github.io/msoanames/MSOA-Names-1.20.csv', ddir) )

In [ ]:
print(msoa_nms.columns.values)
msoa_nms.sample(3, random_state=42)

Now that you've loaded the `msoa_nms` data you need to merge it with our `ldn_msoas`. You will need to deal with the fact that the left and right fields have different names and may also want to think about the `how` of the merge. In *this* case, the result of the merge *should* be a GeoDataFrame, but **this is not always guaranteed** so you may want to double-check or run multiple tests before assuming that you'll get back a geographically aware object.

In [ ]:
msoas = pd.merge(ldn_msoas, msoa_nms, left_on='MSOA11CD', right_on='msoa11cd', how='inner')
print(f"MSOAs shape is {' x '.join([str(x) for x in msoas.shape])}.")
print(f"Resulting class is a {type(msoas).__name__}.") # You should check this -- result isn't always be a GeoDataFrame
msoas.sample(3, random_state=42)[['OBJECTID','MSOA11CD','MSOA11NM','msoa11hclnm']]

Your result should be:

|    | OBJECTID | MSOA11CD | MSOA11NM | msoa11hclnm |
| -: | -------: | -------: | :------- | :---------- |
| **810** | 811 | E02000841 | Sutton 002 | St Helier South |
| **801** | 802 | E02000832 | Southwark 026 | Nunhead North |
| **813** | 814 | E02000844 | Sutton 005 | The Wrythe |

#### 1.3.3. Map

Set up a `mapping` dict here so that you can apply it as part of the `groupby` operation below (you should have 33 keys when done):

In [ ]:
mapping = {}
for b in ['Enfield','Waltham Forest','Redbridge','Barking and Dagenham','Havering','Greenwich','Bexley']:
    mapping[b]='Outer East and North East'
for b in ['Haringey','Islington','Hackney','Tower Hamlets','Newham','Lambeth','Southwark','Lewisham']:
    mapping[b]='Inner East'
for b in ['Bromley','Croydon','Sutton','Merton','Kingston upon Thames']:
    mapping[b]='Outer South'
for b in ['Wandsworth','Kensington and Chelsea','Hammersmith and Fulham','Westminster','Camden']:
    mapping[b]='Inner West'
for b in ['Richmond upon Thames','Hounslow','Ealing','Hillingdon','Brent','Harrow','Barnet','City of London']:
    mapping[b]='Outer West and North West'
print(len(mapping.keys()))

In [ ]:
msoas['Subregion'] = msoas.Borough.map(???)

#### 1.3.3. Tidy Up

In [ ]:
msoas.columns.to_list()

In [ ]:
to_drop = ['MSOA11CD','MSOA11NM','MSOA11NMW','LONG','LAT','Shape__Are','Shape__Len','index_right',
           'Laname','msoa11hclnmw','msoa11nmw']
msoas.drop(columns=to_drop, inplace=True)
print(msoas.shape)

#### 1.3.4. And Save

In [ ]:
msoas.to_parquet(os.path.join('data','geo','London_MSOA_Names.geoparquet'))

### Task 1.4: Load InsideAirbnb Data

In [ ]:
host = 'http://orca.casa.ucl.ac.uk'
path = '~jreades/data'
listings = gpd.read_parquet( cache_data(f'{host}/{path}/2023-09-06-listings.geoparquet', ddir) )
listings = listings.to_crs(epsg=27700)
print(f"Data frame is {listings.shape[0]:,} x {listings.shape[1]}")

In [ ]:
listings = listings.to_crs('epsg:27700')

#### 1.4.1. Spatial Join

Associate LA (Local Authority) names to the listings using a spatial join, but **notice** the `how` here:

In [ ]:
gdf_la = gpd.sjoin(listings, ???, predicate='???', how='left')

In [ ]:
print(gdf_la.columns.to_list())

#### 1.4.2. Tidy Up

In [ ]:
gdf_la.drop(columns=['index_right','HECTARES','NONLD_AREA','ONS_INNER'], inplace=True)

You'll need to look closely to see what's wrong in the `value_counts` output, but here's a clue:

In [ ]:
print(f"Now there are... {len(gdf_la.NAME.unique())} boroughs?")

In [ ]:
gdf_la.NAME.value_counts(dropna=False)

In [ ]:
gdf_la.columns

In [ ]:
gdf_la[gdf_la.NAME.isna()].sample(2)[['name', 'NAME']]

#### 1.4.3. Find Problematic Listings

Let's map them and see if you can work out what happened...

In [ ]:
ax = gdf_la[gdf_la.NAME.isna()].plot(figsize=(10,7), markersize=5, alpha=0.5)
boros.plot(ax=ax, edgecolor='r', facecolor='None', alpha=0.5);

<img src="https://github.com/jreades/fsds/blob/master/practicals/img/Unmatched_Listings.png?raw=true" alt="Unmatched Listings" style="width:50%" />

#### 1.4.4. Drop Problematic Results

In [ ]:
gdf_la.drop(index=gdf_la[gdf_la.NAME.isna()].index, axis=1, inplace=True)
print(f"Data frame is {gdf_la.shape[0]:,} x {gdf_la.shape[1]}")

You should now have 67,993 records.

#### 1.4.5. Check

In [ ]:
ax = gdf_la.plot(column='NAME', markersize=0.5, alpha=0.5, figsize=(10,8))
boros.plot(ax=ax, edgecolor='r', facecolor='None', alpha=0.5);

#### 1.4.6. Save

In [ ]:
gdf_la.to_parquet(os.path.join('data','geo','Listings_with_LA.geoparquet'))

#### 1.4.7. Questions

- Do you understand the difference between `how='inner'` and `how='left'`? 

> 

### Task 1.5: Create LA Data

#### 1.5.1. Select LA

Select a LA that is relevant to _you_ to explore further...

In [ ]:
LA = 'Waltham Forest'

#### 1.5.2. Spatial Join

The first thing we want to do is join MSOA identifiers to each listing. In both cases we want to constrain the data to only be for 'our' LA of interest: 

In [ ]:
msoadf  = gpd.sjoin(
            gdf_la[gdf_la.NAME==LA].reset_index(), 
            msoas[msoas.Borough==LA], predicate='within')

#### 1.5.3. Aggregate

Now aggregate the data by MSOA, deriving median price and a count of the listings:

In [ ]:
msoagrdf = msoadf.groupby('msoa11nm').agg({'price':['median','count']}).reset_index()

In [ ]:
msoagrdf.sample(3, random_state=42)

You should get something like the below (if you're using Waltham Forest):

|   | msoa11nm |  | price |
| -: | ------: | ----: | ----: |
|   | **median** | **count** |
| 9 | Waltham Forest 010 | 55.0 | 25 |
| 25 | Waltham Forest 026 | 75.0 | 72 |
| 8 | Waltham Forest 009 | 51.5 | 26 |

#### 1.5.4. Resolve Columns

Which level value is easier to use? 0? or 1?

In [ ]:
msoagrdf.columns = msoagrdf.columns.get_level_values(1)
msoagrdf.head()

Fix the missing column name:

In [ ]:
msoagrdf.rename(columns={'':'msoa11nm', 'count':'listings'}, inplace=True)
msoagrdf.head()

#### 1.5.5. Join (Again)

Here we see the **difference between merge and join**. You'll notice that `join` operates by taking one data frame as the implicit '*left*' table (the one which *calls* join) while the one that is passed to the join function is, implicitly, the '*right*' table. Join operates only using indexes, so you'll need to insert the code to specify the same index on both data frames, but this can be done **on-the-fly** as part of the joining operation:

In [ ]:
msoa_gdf = msoagrdf.set_index('msoa11nm').join(
                msoas[msoas.Borough==LA].set_index('msoa11nm'), 
                rsuffix='_r')
msoa_gdf.head(3)

#### 1.5.6. Resolve Geodata

You need to add a command in order to help python recognise that this should be a GeoDataFrame:

In [ ]:
msoa_gdf = msoa_gdf.set_geometry('geometry')

In [ ]:
msoa_gdf.plot(column='median', legend=True, figsize=(8,8));

#### 1.5.7. Save

Just so that we can pick up here without having to re-run all the preceding cells.

In [ ]:
msoa_gdf.to_parquet(os.path.join('data','geo',f'{LA}-MSOA_data.geoparquet'))

#### 1.5.8. Questions

- Do you understand the differences between `pd.merge` and `df.join`? and `gpd.sjoin`?

> 

- Do you understand why it may be necessary to `set_geometry` in some cases?

> 

## Task 2. Using Maplotlib

### Task 2.1: Anatomy of a Figure

<div class="alert alert-block alert-success">
    <b>&#128161; Tip</b>: You might want to bookmark the 'Anatomy of a Figure' image so that you can easily find and refer to it in the future. This structure is why <tt>matplotlib</tt> is so much nastier than <tt>ggplot</tt>, but it does also give you greater <em>control</em> over the output if you really dig into the guts of things.
</div>

*One* of the reasons that Matplotlib is so much more complex than `ggplot` is that it can actually *do* a plot more than ggplot, including image manipulation, axis translation, and even 3D. You can get a sense of this by looking at [the tutorials](https://matplotlib.org/stable/tutorials/index.html) since the [Users guide](https://matplotlib.org/stable/users/index.html) can be a bit overwhelming.

Nevertheless, the [core components of all matplotlib figures](https://matplotlib.org/stable/gallery/showcase/anatomy.html) can be seen here:

<img src="https://matplotlib.org/stable/_images/sphx_glr_anatomy_001.png" alt="Anatomy of a Figure" style="width:50%" />

### Task 2.2: Finding Fonts

I find matplotlib's use of fonts to be _profoundly_ weird. If you use `conda` and install directly on to the computer then you _might_ have access to all of your computer's fonts (though there are different *types* of fonts as well, not all of which will show up), but for most users it will be those that were installed into Docker. 

#### 2.2.1. Using Fontconfig

`fontconfig` is the base Linux utility for managing fonts. We can list font using `fc-list` and then a set of 'switches' determining the kind of information we want back. Since fontconfig doesn't exist on OSX or Windows, you'll need to do some more investigating and poking around to get these details on a `conda` install (I'll show an option further down)...

<div class="alert alert-block alert-success">
    <b>&#128161; Tip</b>: notice that below we're able to capture the output of an external application (called via the Terminal) with <tt>fonts = ! ...</tt>. This can be useful when something is easy to do on the command line but hard to do in Python.
</div>

Here we ask fontconfig to format the output so that we only get the first part of the family name, and then we pipe (recall `|` sends output from one utility to another!) the output of that to `sort`, which sorts the output, and `uniq` which removes duplicates (which there will be because there are **bold**, *italic*, small-caps, etc. versions of each font). To make better sense of this you can always try playing around with all three steps in the output below!

In [ ]:
fonts = ! fc-list --format="%{family[0]}\n" | sort | uniq
print(fonts)

This also pipes output from fonctconfig, but to the `grep` utility which checks each line for the character sequence `Liberation`. Now we're asking fontconfig to include `style` details which will relate to both weight (regular, bold, extra bold, light, etc.) and italic, bold, small caps, etc.

In [ ]:
fonts = ! fc-list : family style | grep "Liberation"
print(sorted(fonts))

You can find more examples [here](https://www.geeksforgeeks.org/fc-list-command-in-linux-with-examples/), a more detailed set of instructions [here](https://www.freedesktop.org/software/fontconfig/fontconfig-user.html), and even information about (for example) [supported languages](https://thottingal.in/blog/2016/03/04/fontconfig-language-matching/) based on [RFC 3066](http://www.i18nguy.com/unicode/language-identifiers.html).

Here are the languages supported by the Ubuntu Light font:

In [ ]:
! fc-list "Ubuntu Light" : lang

Here are the monospace fonts installed: 

In [ ]:
! fc-list :spacing=mono : family | sort | uniq

#### 2.2.2. Using Python+Terminal

Another way to get at this information is to try asking matplotlib what fonts it *already* knows about in its cache:

In [ ]:
import matplotlib
loc = matplotlib.get_cachedir()
!ls {loc}

Hopefully you will see a list of installed fonts when you run this next block of code. See if you can make sense of what this code does!

In [ ]:
fonts = !cat {loc + '/fontlist-v330.json'}
fonts = set(list(filter(lambda x:'"name"' in x, fonts)))
fonts = [x.replace('      "name": "','').replace('",','') for x in fonts]
print(fonts)

In [ ]:
fonts = ! fc-list : family style | grep "Ubuntu"
print(sorted(fonts))

#### 2.2.3. Fontdicts

Now that we know what's available, the next step is to set up some useful defaults that we can re-use across multiple plots to ensure consistency of output. The format for specifying fonts on a per-figure basis is a dictionary, so where you see `fontdict` in the `matplotlib` [documentation](https://matplotlib.org/stable/gallery/text_labels_and_annotations/text_fontdict.html) the following should work:

Here's the example:

```python
font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 16,
        }
```

In [ ]:
tfont = {'fontname':'Liberation Sans Narrow', 'weight':'bold'}
bfont = {'fontname':'Liberation Sans Narrow', 'weight':'normal', 'horizontalalignment':'left'}
afont = {'fontname':'Liberation Sans Narrow', 'weight':'light'}

I am setting the 'title font' (`tfont`) and 'body copy font' (`bfont`) and 'axis font' (`afont`) here to use in the output below. You can pick another font and see what happens. 

### 2.3: Using Fonts

At this point we're going to work towards a kind of 'atlas' that would make it easy to compare some features for different London boroughs. I basically implemented a the basic `matplotlib` version of QGIS' Atlas functionality.

In [ ]:
LA = 'Waltham Forest'

In [ ]:
msoa_gdf = gpd.read_parquet(os.path.join('data','geo',f'{LA}-MSOA_data.geoparquet'))

In [ ]:
median_gdf  = msoa_gdf[['msoa11cd','median','geometry']]
listing_gdf = msoa_gdf[['msoa11cd','listings','geometry']].rename(columns={'listings':'count'})

In [ ]:
import matplotlib.pyplot as plt

#### 2.3.1. The Defaults

Here is a demonstration of some of the ways you can adjust features in a Python matplotlib plot. I'm not suggesting either of these is a _good_ output, but that's not the point! The idea is to see the various ways you can tweak a plot... And notice that we've not yet changed any fonts. And it shows.

In [ ]:
# Set up a 1 x 2 plot (you can also leave off the nrows= and ncols=)
f,axes = plt.subplots(nrows=1, ncols=2, figsize=(10,8))
# ax1 will be the first plot on the left, ax2 will be on the right;
# a 2 (or more) *row* plot will return a list of lists... 1 list/row.
ax1 = axes[0]
ax2 = axes[1]

# Left plot is the median price
median_gdf.plot(column='median', ax=ax1, legend=True, cmap='viridis')
ax1.set_title("Median Price per MSOA");
# Turn off the frame, one side of the plat at a time
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.spines['bottom'].set_visible(False)
ax1.spines['left'].set_visible(False)
# Set the labels
ax1.set_xlabel("Easting");
ax1.set_ylabel("Northing");

# Right plot is the number of listings; note
# here the use of both zorder (which is the 
# 'stacking order' of elements on the plot, and
# the legend_kwds (keywords) to change the 
# orientation of the plot to horizontal
listing_gdf.plot(column='count', ax=ax2, legend=True, cmap='plasma', zorder=1, 
                 legend_kwds={"orientation": "horizontal"})
ax2.set_title("Count of Listings per MSOA");
# Set a background colour for the plot
ax2.set_facecolor((.4, .4, .4, .2))
# Add grid lines and set their zorder to
# below that of the data on the plot
plt.grid(visible=True, which='major', axis='both', color='w', linestyle='-', linewidth=2, zorder=0)
ax2.set_axisbelow(True)

# This is equivalent to the ax1.spines... 
# above, but if you use it here you lose
# the background to the plot as well!
#plt.gca().set(frame_on=False)

# Remove the labels on the ticks of the 
# axes (meaning: remove the numbers on 
# x- and y-axes).
ax2.set_xticklabels([])
ax2.set_yticklabels([])

# Set the labels
ax2.set_xlabel("Easting");
ax2.set_ylabel("Northing");

#### 2.3.2: Improving on Defaults

In [ ]:
f,axes = plt.subplots(1,2,figsize=(12,8))

# Set up the plots
median_gdf.plot(column='median', ax=axes[0], legend=True, cmap='viridis')
listing_gdf.plot(column='count', ax=axes[1], legend=True, cmap='plasma')
for ax in axes:
    ax.axis('off')
    # Note that here, set_facebolor doesn't work,
    # presumably because the axis is 'off'
    ax.set_facecolor((.4, .4, .4, .2))

# Add the 'super-title', but notice that it is not 
# longer either centered (x=0.025) or centre-aligned
# (horizonal alignment=left). We also see **tfont, which
# is a way of expading the 'tfont' dictionary into a 
# set of parameters to a function call. We do the same
# for the titles on each figure, but passing a different
# fontdict.
f.suptitle(LA, x=0.025, ha='left', size=24, **tfont)
axes[0].set_title('Median Price', size=20, **afont)
axes[1].set_title('Count', size=20, **afont)

# And add a short piece of text below the borough
plt.figtext(x=0.025, y=0.92, linespacing=1.4, va='top', size=12, 
            s=f"Total listings: {listing_gdf['count'].sum():,.0f}\nMedian price: ${median_gdf['median'].median():,.2f}", **bfont);

## Task 3. Create an Atlas

### Task 3.1: Adding Picture-in-Picture 

We're now going to emulate a _bit_ of QGIS' Atlas function by creating two subplots and then adding a _third_ plot afterwards that shows where the borough is.

In [ ]:
f,axes = plt.subplots(1,3,gridspec_kw={'width_ratios':[1,4,4]}, figsize=(12,8))

# Plot 0 is basically being used as a 'spacer' 
# as you'll see below
axes[0].axis('off')

# Plot 1 is the median price
median_gdf.plot(column='median', ax=axes[1], legend=True, cmap='viridis')
axes[1].set_title('Median Price', size=20, **afont)

# Plot 2 is the count of listings
listing_gdf.plot(column='count', ax=axes[2], legend=True, cmap='plasma')
axes[2].set_title('Count', size=20, **afont)

# For plots 1 and 2... if you were doing this a lot it could be a function!
for ax in axes[1:]:
    ax.set_facecolor((.9, .9, .9, .5))
    ax.grid(visible=True, which='major', axis='both', color='w', linestyle='-', linewidth=2, zorder=0)
    ax.set_axisbelow(True)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.tick_params(axis='both', which='both', length=0)

# Add a *third* chart that we use as a kind of 'PiP'
# to show which borough we're talking about. The 
# add_axes call is here taking information about the
# positioning and size of the additional figure.
# Disable ax2.axis('off') if you want to see the
# figure in full.
ax3 = f.add_axes([0.015, 0.7, 0.2, 0.2])
boros.plot(facecolor='lightgrey', edgecolor='w', linewidth=1, ax=ax3)
boros[boros.NAME==LA].plot(facecolor='r', edgecolor='none', hatch='///', ax=ax3)
ax3.axis('off')

# Add the 'super-title', but notice that it is not 
# longer either centered (x=0.025) or centre-aligned
# (horizonal alignment=left). We also see **tfont, which
# is a way of expading the 'tfont' dictionary into a 
# set of parameters to a function call. We do the same
# for the titles on each figure, but passing a different
# fontdict.
f.suptitle(LA, x=0.025, ha='left', size=24, **tfont)

# And add a short piece of text below the borough
plt.figtext(x=0.025, y=0.65, s=f"Total listings: {listing_gdf['count'].sum():,.0f}", size=12, **bfont);

### Task 3.2: Bonus Achievement Unlocked!

If you have the time and inclination, see if you can convert the above to an _actual_ atlas output: 

1. You'll want to turn this plot into a function so as to be able to produce (and save) the map for _every_ borough. 
2. You'll even need to parameterise the filename so that you save to _different_ PNG files as well as going back to see how we generated the listing and pricing data frames for the Local Authority... 
3. And you'll *also* need to make sure that you ensure a consistent colour bar (for all of London, because the median price and number of listings will vary rather a lot by LA)
4. Then there's the placement of the PiP for some boroughs with long names
5. And finally, you might consider adding some more text to atlas--maybe pull some content from Wikipedia using Beautiful Soup (`bs4`)?

## Task 4. Think Text!

I also wanted to draw your attention to this [outstanding piece](https://blog.datawrapper.de/text-in-data-visualizations/) on using text effectively in data visualisation: we often add labels as afterthoughts without too much regard for where they go or how they look; however, getting the content, positioning, size, and even font/font-weight 'right' can make all the difference to the effectiveness of your chart! The illustrations are top-notch.

And see the bibliography at the end!

<div class="alert alert-block alert-success">
    <b>&#128161; Tip</b>: Basically, bookmark this blog post and refer to it every time you are making a map or chart.
</div>

## Task 5. Using Bokeh

Bokeh can do a *lot* more than this, but I just wanted to give you a flavour of the other visualisation tools supported by Python. This obviously works *very* differently in setup and use.

In [ ]:
LA = 'Camden'

In [ ]:
gdf_la = gpd.read_parquet(os.path.join('data','geo','Listings_with_LA.geoparquet'))
msoas  = gpd.read_parquet(os.path.join('data','geo','London_MSOA_Names.geoparquet'))

### Task 5.1: For a Chart

Group the listings by Borough and Room Type, and aggregate by median price, also producing a count variable for the number of listings of each type in each Borough.

In [ ]:
la_tots = gdf_la[gdf_la.NAME==LA].groupby(by='room_type', observed=False).agg(
                            {'price':'median', 'listing_url':'count'}
                        ).reset_index().rename(columns={'listing_url':'count'})
la_tots

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Spectral4
from bokeh.models import CustomJS, Dropdown

output_notebook()

room_types = la_tots.room_type.to_list()
prices     = la_tots.price.to_list()
counts     = la_tots['count'].to_list()

# Add hover tooltip
source = ColumnDataSource(data=dict(
    rt=room_types,
    count=counts,
    price=prices,
))

TOOLTIPS = [
    ("Room Type", "@rt"),
    ("Number of Listings", "@count{,}"),
    ("Median Price", "$@price{,}/night")
]

p = figure(x_range=room_types, height=300, tooltips=TOOLTIPS,
           title=f"Median Price by Room Type in {LA}",
           toolbar_location=None, tools="")

p.vbar(x='rt', top='count', width=0.9, source=source)
p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)

### Task 5.2: For a Map

This is not the prettiest code, but it should work...

In [ ]:
from bokeh.plotting import figure
from bokeh.tile_providers import CARTODBPOSITRON, get_provider

from bokeh.io import output_file, show, output_notebook, push_notebook, export_png
from bokeh.models import ColumnDataSource, GeoJSONDataSource, LinearColorMapper, ColorBar, HoverTool
from bokeh.plotting import figure
from bokeh.palettes import brewer

output_notebook()

In [ ]:
msoadf = gpd.sjoin(
            gdf_la[gdf_la.NAME==LA].reset_index(), 
            msoas[msoas.Borough==LA], predicate='within')

In [ ]:
msoagrdf = msoadf.groupby('msoa11nm').agg({'price':['median','count']}).reset_index()
msoagrdf.columns=['msoa11nm','median','count']

I cobbled the mapping functions below together from two tutorials I found online ([this one](https://github.com/dmnfarrell/teaching/blob/master/geo/maps_python.ipynb) and [this one](https://widdowquinn.github.io/Teaching-Data-Visualisation/exercises/interactive_bokeh_map/interactive_bokeh_map.html)). As you can see, this is a very different approach to mapping data, but it has clear benefits for exploratory purposes and produces fast, interactive maps... and I've not even added selection and filtering tools!

In [ ]:
def get_geodatasource(gdf):    
    """Get getjsondatasource from geopandas object"""
    json_data = json.dumps(json.loads(gdf.to_json()))
    return GeoJSONDataSource(geojson = json_data)

def bokeh_plot_map(gdf, column=None, title=''):
    """Plot bokeh map from GeoJSONDataSource """
    
    tile_provider = get_provider(CARTODBPOSITRON)

    geosource = get_geodatasource(gdf)
    palette = brewer['OrRd'][8]
    palette = palette[::-1]
    vals = gdf[column]
    
    #Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
    color_mapper = LinearColorMapper(palette=palette, low=vals.min(), high=vals.max())
    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=500, height=10,
                         location=(0,0), orientation='horizontal')

    tools = 'wheel_zoom,pan,reset,hover'
    
    p = figure(title = title, height=700, width=850, toolbar_location='right', tools=tools)
    p.add_tile(tile_provider)
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    
    # Add patch renderer to figure
    p.patches('xs','ys', source=geosource, fill_alpha=0.5, line_width=0.5, line_color='white',  
              fill_color={'field' :column , 'transform': color_mapper})
    
    # Specify figure layout.
    p.add_layout(color_bar, 'below')
    
    # Add hover
    hover = p.select_one(HoverTool)
    hover.point_policy = "follow_mouse"
    hover.tooltips = [("Borough", "@Borough"),
                      ("Neighbourhood", "@msoa11hclnm"),
                      ("Count of Listings", "@count"),
                      ("Median Price", "$@median")]
    
    return p

Reproject to Web Mercator:

In [ ]:
msoa_gdf = pd.merge(msoagrdf, msoas, left_on='msoa11nm', right_on='msoa11nm', how='inner')
msoa_gdf = msoa_gdf.set_geometry('geometry').set_crs('epsg:27700')

In [ ]:
msoageo = msoa_gdf.to_crs('epsg:3785')
msoageo.total_bounds

And map it!

In [ ]:
p = bokeh_plot_map(msoageo, 'median', title=f'MSOA-Level Activity in {LA}')

handle = show(p, notebook_handle=True)
push_notebook(handle=handle)

<div class="alert alert-block alert-info">
    <b>&#128279; Connections</b>: And that's it. That's all she wrote! You've now covered in 10 weeks what many people might take 10 <em>months</em> to cover. So do not feel like either: 1) you know it all; or 2) you know nothing. You have learned a <em>lot</em>, but it's probably just enough to see how much you <em>don't</em> know. That's the start of wisdom. Good luck, young Python-master!
</div>